In [ ]:
# | default_exp routes.datacenter

In [ ]:
# | exporti
import httpx
from enum import Enum

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
class Datacenter_Enum(Enum):
    DATASET = "DATASET"
    DATAFLOW = "DATAFLOW"
    ACCOUNT = "ACCOUNT"


class Dataflow_Type_Filter_Enum(Enum):
    ADR = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ADR",
        "name": "ADR",
        "not": False,
    }

    MYSQL = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    REDSHIFT = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    MAGICV2 = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MAGIC",
        "name": "Magic ETL v2",
        "not": False,
    }

    MAGIC = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ETL",
        "name": "Magic ETL",
        "not": False,
    }

In [ ]:
# | export
def generate_search_datacenter_body(
    entities_list: list[str] = ["DATASET"],
    filters: list[dict] = None,
    combineResults: bool = True,
    count: int = 10,
    offset: int = 0,
):
    return {
        "entities": entities_list,
        "filters": filters or [],
        "combineResults": combineResults,
        "query": "*",
        "count": count,
        "offset": offset,
    }

In [ ]:
# | export
def generate_search_datacenter_account_body(
    search_str: str, is_exact_match: bool = True
):
    return {
        "count": 100,
        "offset": 0,
        "combineResults": False,
        "query": search_str if is_exact_match else f"*{search_str}*",
        "filters": [],
        "facetValuesToInclude": [
            "DATAPROVIDERNAME",
            "OWNED_BY_ID",
            "VALID",
            "USED",
            "LAST_MODIFIED_DATE",
        ],
        "queryProfile": "GLOBAL",
        "entityList": [["account"]],
        "sort": {"fieldSorts": [{"field": "display_name_sort", "sortOrder": "ASC"}]},
    }

In [ ]:
# | export
async def search_datacenter(
    auth: dmda.DomoAuth,
    loop_until_end: bool = True,
    maximum: int = None,
    body: dict = None,
    entity_ls: list[str] = ["DATASET"],
    arr_fn: callable = None,
    limit=100,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    if not body:
        body = generate_search_datacenter_body(
            entities_list=entity_ls,
            filters=[],
            combineResults=False,
        )

    if not arr_fn:

        def arr_fn(res):
            return res.response.get("searchObjects")

    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"

    res = await gd.looper(
        auth=auth,
        session=session,
        url=url,
        body=body,
        offset_params={"offset": "offset", "limit": "count"},
        arr_fn=arr_fn,
        method="POST",
        offset_params_in_body=True,
        limit=limit,
        loop_until_end=loop_until_end,
        debug_api=debug_api,
    )

    return res

#### sample implementation of generate_search body and search datacenter


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

body = generate_search_datacenter_body(
    entities_list=[Datacenter_Enum.DATAFLOW.value],
    filters=[Dataflow_Type_Filter_Enum.MAGICV2.value],
)

res = await search_datacenter(auth=token_auth, body=body)

pd.DataFrame(res.response[0:5])

,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,owners,ownersLocalized,paused,statusOrderPriority,inputCount,outputCount,winnerText,ownedByType,customer,pausedDate
0,dataflow,120,"{'indexName': None, 'databaseId': '120', 'cust...",1673643881000,1673643895000,1673644936883,{},English,False,56.957905,...,"[{'id': '55874022', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Grant Smith', 'count': 1}",False,70,1,1,AT AT Formula Testing,USER,mmmm-0012-0200,NaN
1,dataflow,102,"{'indexName': None, 'databaseId': '102', 'cust...",1664814194000,1674570285000,1674570643197,{},English,False,56.957905,...,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson', 'count': 1}",False,70,1,4,JW_fcst,USER,mmmm-0012-0200,NaN
2,dataflow,115,"{'indexName': None, 'databaseId': '115', 'cust...",1672555986000,1673142746000,1673142746876,{},English,False,55.957905,...,"[{'id': '68216396', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Elliott Leonard', 'count...",False,70,1,1,Test,USER,mmmm-0012-0200,NaN
3,dataflow,121,"{'indexName': None, 'databaseId': '121', 'cust...",1674773199000,1674774797000,1675365424802,{},English,False,55.957905,...,"[{'id': '68216396', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Elliott Leonard', 'count...",False,70,3,1,Alert Effectiveness,USER,mmmm-0012-0200,1.674774e+12
4,dataflow,37,"{'indexName': None, 'databaseId': '37', 'custo...",1619005428000,1619007232000,1619080673093,{},English,False,10.000001,...,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson', 'count': 1}",False,70,2,1,EX01_FuzzyMatch_Master,USER,mmmm-0012-0200,NaN


In [ ]:
# | export
async def get_lineage_upstream(
    auth: dmda.DomoAuth,
    entity_type: str,
    entity_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/lineage/{entity_type}/{entity_id}"

    params = {"traverseDown": "false"}

    return await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
    )

In [ ]:
import os
# import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance='domo-dojo',
    domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

res = await get_lineage_upstream(auth=token_auth,
                                 entity_type = Datacenter_Enum.DATAFLOW.value,
                                 entity_id=4

                                 )

pd.DataFrame([ obj for obj in res.response.values()])

,type,id,descendantCounts,ancestorCounts,complete,children,parents
0,DATA_SOURCE,45a7a24e-c738-44f9-9019-00d5a33613ae,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 2}",True,"[{'type': 'DATAFLOW', 'id': '4', 'complete': T...","[{'type': 'DATAFLOW', 'id': '3', 'complete': T..."
1,DATAFLOW,4,{},"{'DATA_SOURCE': 3, 'DATAFLOW': 2}",True,[],"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-..."
2,DATA_SOURCE,0647a9ed-1a8e-420e-bd66-34f313a18595,{},"{'DATA_SOURCE': 1, 'DATAFLOW': 1}",True,"[{'type': 'DATAFLOW', 'id': '3', 'complete': T...","[{'type': 'DATAFLOW', 'id': '2', 'complete': T..."
3,DATAFLOW,2,{},{'DATA_SOURCE': 1},True,"[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-...","[{'type': 'DATA_SOURCE', 'id': '241025d7-3cca-..."
4,DATA_SOURCE,241025d7-3cca-4369-b7c0-b3264277c0e1,{},{},True,"[{'type': 'DATAFLOW', 'id': '2', 'complete': T...",[]
5,DATAFLOW,3,{},"{'DATA_SOURCE': 2, 'DATAFLOW': 1}",True,"[{'type': 'DATA_SOURCE', 'id': '45a7a24e-c738-...","[{'type': 'DATA_SOURCE', 'id': '0647a9ed-1a8e-..."
